# Simple Storm Surge Inundation Modeling

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_


In this example we will use some of GRASS GIS hydrology tools, namely:

* [r.lake](https://grass.osgeo.org/grass-stable/manuals/r.lake.html): to fill a lake to a target water level from a given start point or seed raster
* [r.lake.series](https://grass.osgeo.org/grass-stable/manuals/r.lake.series.html): addon which runs r.lake for different water levels


In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("nags_head/PERMANENT");
# gj.init("nags_head/flooding");

In [ ]:
# Make a new mapset for this part
gs.run_command("g.mapset", mapset="flooding", location="nags_head", flags="c")

In [ ]:
!g.region region=jockeys_ridge w=w+1200 -p

In [ ]:
!g.extension extension=r.lake.series

In [ ]:
flood1 = gj.Map(use_region=True)
flood1.d_rast(map="JR_2014")
flood1.show()

In [ ]:
!r.neighbors input=JR_2014 output=JR_2014_filled method=average size=9

!r.mapcalc "JR_2014_filled_bath = if(isnull(JR_2014_filled), -5.0, JR_2014_filled)"

In [ ]:
!r.colors map=JR_2014_filled_bath color=elevation

flood1 = gj.Map()
flood1.d_rast(map="JR_2014_filled_bath")
flood1.show()

In [ ]:
fill_point = "914151,250381"

gs.write_command("v.in.ascii", input="-", output="fill_point", separator="comma", stdin=fill_point)

In [ ]:
flood1 = gj.Map()
flood1.d_rast(map="JR_2014_filled_bath")
flood1.d_vect(map="fill_point", icon="basic/pin", size=20, fill_color="red")
flood1.show()

In [ ]:
gs.run_command("r.lake.series",
                elevation="JR_2014_filled_bath",
                output="flooding",
                start_water_level=3.0,
                end_water_level=4.0,
                water_level_step=0.25,
                coordinates=fill_point
              )

In [ ]:
from io import StringIO
import pandas as pd

pd.read_csv(StringIO(gs.read_command("t.rast.list", input="flooding", separator="comma")))

r.lake.series creates a space-time dataset. We can use the [temporal modules](https://grass.osgeo.org/grass-stable/manuals/temporal.html) to further work with the data.

Let's visualize the results:

In [ ]:
flood1 = gj.Map()
flood1.d_rast(map="flooding_4.0")
flood1.show()

In [ ]:
!r.relief input=JR_2014_filled_bath output=JR_2014_shade

In [ ]:
gs.run_command("t.rast.colors", input="flooding", color="water")

timemap = gj.TimeSeriesMap()
timemap.d_rast(map="JR_2014_shade")
timemap.add_raster_series("flooding")
timemap.d_legend(at=(1, 40, 90, 95))
#timemap.show() 

In [ ]:
timemap.save(filename="flood.gif");

In [ ]:
from IPython.display import display, Image

display(Image(filename="flood.gif"))